In [18]:
import pandas as pd
import numpy as np
import matplotlib.pylab as plt
from itertools import cycle
import seaborn as sns
import kagglehub
import os
import cv2
from glob import glob
import librosa 
import librosa.display
import IPython.display as ipd 

In [19]:
sns.set_theme(style="white", palette=None)
color_pal = plt.rcParams["axes.prop_cycle"].by_key()["color"]
color_cycle = cycle(plt.rcParams["axes.prop_cycle"].by_key()["color"])


In [20]:
test_mp3_files = glob(r'../ml_music/test_data/*.mp3')

y, sr = librosa.load(test_mp3_files[0])

In [21]:
def extract_mel_spectrogram(file_path, target_size=(128, 128)):
    audio, sr = librosa.load(file_path, sr=22050)
    mel_spec = librosa.feature.melspectrogram(y=audio, sr=sr, n_mels=128)
    mel_spec_db = librosa.power_to_db(mel_spec, ref=np.max)
    
    # Normalize and resize
    mel_spec_db = (mel_spec_db - mel_spec_db.min()) / (mel_spec_db.max() - mel_spec_db.min())
    mel_spec_resized = cv2.resize(mel_spec_db, target_size)
    
    # Add channel dimension
    return mel_spec_resized[..., np.newaxis]

In [22]:
spectrograms = [extract_mel_spectrogram(file_path) for file_path in test_mp3_files]

In [23]:
Training_files = glob(r'../ml_music/moodsClassed/*.mp3')

In [24]:
Training_files

['../ml_music/moodsClassed\\Chill.mp3',
 '../ml_music/moodsClassed\\Chill1.mp3',
 '../ml_music/moodsClassed\\CHill2.mp3',
 '../ml_music/moodsClassed\\Chillbeat2.mp3',
 '../ml_music/moodsClassed\\Chillyy.mp3',
 '../ml_music/moodsClassed\\Chill_Lofi.mp3',
 '../ml_music/moodsClassed\\enenenrgy.mp3',
 '../ml_music/moodsClassed\\Energetic33.mp3',
 '../ml_music/moodsClassed\\Energy.mp3',
 '../ml_music/moodsClassed\\Energy3.mp3',
 '../ml_music/moodsClassed\\Energyyyy.mp3',
 '../ml_music/moodsClassed\\Relax.mp3',
 '../ml_music/moodsClassed\\Relaxblud.mp3',
 '../ml_music/moodsClassed\\Relaxing.mp3',
 '../ml_music/moodsClassed\\Relaxing2.mp3',
 '../ml_music/moodsClassed\\Upbeat1.mp3',
 '../ml_music/moodsClassed\\Upbeat2.mp3',
 '../ml_music/moodsClassed\\upbeat22.mp3',
 '../ml_music/moodsClassed\\Upbeat3.mp3',
 '../ml_music/moodsClassed\\Upbeatyy.mp3']

In [25]:
categories = {
    "1": ["chill"],
    "3": ["energy", "energetic", "enenenrgy", "enerrgy"],
    "0": ["relax", "relaxing"],
    "2": ["upbeat", "uppy"]
}


In [26]:
def classify_file(file_name, categories):
    file_name_lower = file_name.lower()  # Make case-insensitive
    for category, keywords in categories.items():
        if any(keyword in file_name_lower for keyword in keywords):
            return category
    return "Unknown"  # If no keyword matches

# Create a dictionary of file paths and their categories
classified_files = {os.path.basename(file): classify_file(file, categories) for file in Training_files}

In [27]:
classified_files

{'Chill.mp3': '1',
 'Chill1.mp3': '1',
 'CHill2.mp3': '1',
 'Chillbeat2.mp3': '1',
 'Chillyy.mp3': '1',
 'Chill_Lofi.mp3': '1',
 'enenenrgy.mp3': '3',
 'Energetic33.mp3': '3',
 'Energy.mp3': '3',
 'Energy3.mp3': '3',
 'Energyyyy.mp3': '3',
 'Relax.mp3': '0',
 'Relaxblud.mp3': '0',
 'Relaxing.mp3': '0',
 'Relaxing2.mp3': '0',
 'Upbeat1.mp3': '2',
 'Upbeat2.mp3': '2',
 'upbeat22.mp3': '2',
 'Upbeat3.mp3': '2',
 'Upbeatyy.mp3': '2'}

In [28]:
spectrogram_train = []  # List to store spectrograms
labels = []  
for file in Training_files:
    try:
        # Extract spectrogram
        spectrogram = extract_mel_spectrogram(file)
        
        # Get classification
        label = classify_file(file, categories)
        
        if spectrogram is not None and label != "Unknown":  # Ensure valid data
            spectrogram_train.append(spectrogram)  # Add spectrogram to the list
            labels.append(label)             # Add corresponding label
    except Exception as e:
        print(f"Error processing file {file}: {e}")

In [29]:
labels = np.array(labels)
labels = labels.reshape(-1,)

In [30]:
def plot_sample(x, y, index):
    plt.imshow(x[index])
    plt.xlabel(labels[index])

In [31]:
from tensorflow.keras import layers, models
ann = models.Sequential([
        layers.Flatten(input_shape=(128, 128, 1)),
        layers.Dense(512, activation='relu'),
        layers.Dense(256, activation='relu'),
        layers.Dense(4, activation='softmax')    
    ])

ann.compile(optimizer='SGD',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

C:\Users\jorda\AppData\Roaming\Python\Python312\site-packages\keras\src\layers\reshaping\flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [32]:
spectrogram_train = np.array(spectrogram_train)

In [33]:
labels = np.array(labels, dtype=np.int32)


In [34]:
ann.fit(spectrogram_train, labels, epochs=5)

Epoch 1/5
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 631ms/step - accuracy: 0.3000 - loss: 1.5222
Epoch 2/5
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 96ms/step - accuracy: 0.2500 - loss: 1.9467
Epoch 3/5
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 97ms/step - accuracy: 0.4500 - loss: 1.5631
Epoch 4/5
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 111ms/step - accuracy: 0.4000 - loss: 1.3112
Epoch 5/5
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 116ms/step - accuracy: 0.3500 - loss: 1.4911


In [35]:
cnn = models.Sequential([
    layers.Conv2D(filters=32, kernel_size=(3, 3), activation='relu', input_shape=(128, 128, 1)),
    layers.MaxPooling2D((2, 2)),
    
    layers.Conv2D(filters=64, kernel_size=(3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(10, activation='softmax')
])

cnn.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

C:\Users\jorda\AppData\Roaming\Python\Python312\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [36]:
cnn.fit(spectrogram_train, labels, epochs=10)

Epoch 1/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step - accuracy: 0.2500 - loss: 2.2703
Epoch 2/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 196ms/step - accuracy: 0.3000 - loss: 2.1703
Epoch 3/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 197ms/step - accuracy: 0.4500 - loss: 1.4904
Epoch 4/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 235ms/step - accuracy: 0.4000 - loss: 1.5552
Epoch 5/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 214ms/step - accuracy: 0.3500 - loss: 1.2358
Epoch 6/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 252ms/step - accuracy: 0.5000 - loss: 1.1283
Epoch 7/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 215ms/step - accuracy: 0.6000 - loss: 1.0712
Epoch 8/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 191ms/step - accuracy: 0.7500 - loss: 0.9619
Epoch 9/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 215ms/step - accuracy: 0.9500 - loss: 0.9021
Epoch 10/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 195ms/step - accuracy: 0.8500 - loss: 0.7771


In [38]:
test_mp3_files

['../ml_music/test_data\\10.mp3',
 '../ml_music/test_data\\12.mp3',
 '../ml_music/test_data\\13.mp3',
 '../ml_music/test_data\\17.mp3',
 '../ml_music/test_data\\18.mp3',
 '../ml_music/test_data\\19.mp3',
 '../ml_music/test_data\\2.mp3',
 '../ml_music/test_data\\20.mp3',
 '../ml_music/test_data\\21.mp3',
 '../ml_music/test_data\\22.mp3',
 '../ml_music/test_data\\24.mp3',
 '../ml_music/test_data\\25.mp3',
 '../ml_music/test_data\\3.mp3',
 '../ml_music/test_data\\31.mp3',
 '../ml_music/test_data\\32.mp3',
 '../ml_music/test_data\\35.mp3',
 '../ml_music/test_data\\39.mp3',
 '../ml_music/test_data\\4.mp3',
 '../ml_music/test_data\\40.mp3',
 '../ml_music/test_data\\41.mp3',
 '../ml_music/test_data\\42.mp3',
 '../ml_music/test_data\\43.mp3',
 '../ml_music/test_data\\44.mp3',
 '../ml_music/test_data\\45.mp3',
 '../ml_music/test_data\\46.mp3',
 '../ml_music/test_data\\47.mp3',
 '../ml_music/test_data\\48.mp3',
 '../ml_music/test_data\\5.mp3',
 '../ml_music/test_data\\7.mp3',
 '../ml_music/test_